In [37]:
# DESCRIÇÃO DO EXERCÍCIO 04

# Aluno: Wekler Mendes Sarmento

# Escolha uma base de classificação e compare os classificadores Logistic Regression e KNN do scikit-learn.

# Use pelo menos duas formas de avaliação e as repita pelo menos 10 vezes.

# Calcule a média das repetições de cada avaliação.



#==============================================================================================================================================================
# Importando as dependências do código
#==============================================================================================================================================================
# Instalando o pandas para trabalhar com a leitura de arquivos csv (abrir o terminal e instalar o pandaspip install pandas)
# Para leitura de arquivos xlsx precisa-se instalar no computador local o pacote (pip install xlrd openpyxl)
import pandas as pd # Importando a dependêcia para leitura do csv
from sklearn.metrics import mean_squared_error, make_scorer # Para calcular o MSE e o score
from sklearn.linear_model import LogisticRegression # Importando o algorítimo de classificação
from sklearn.model_selection import RepeatedKFold # Para fazer a divisão de Folds
from sklearn.model_selection import cross_validate # Para utilizar o cross validate
from sklearn.neighbors import KNeighborsClassifier # Para utlizar o KNN para uma base de classificação
from sklearn.preprocessing import StandardScaler # Função de padronização
from sklearn.pipeline import Pipeline # Para utilização do pipeline
from sklearn.model_selection import GridSearchCV # Para utilização do GridSearch
from sklearn import metrics # Para utilização do metrics


####Calculo de pontuação
import numpy as np # [Primeira forma de avaliação] Para o cálculo de precisão
from sklearn.metrics import accuracy_score # [Segunda forma de avaliação] Para o cálculo de precisão

#==============================================================================================================================================================
#BASE DE DADOS
#==============================================================================================================================================================
# Nome: Immunotherapy Dataset Data Set
    # Descrição da base de dados: https://archive.ics.uci.edu/ml/datasets/Immunotherapy+Dataset
    # Link direto para download da base de dados: https://archive.ics.uci.edu/ml/machine-learning-databases/00428/Immunotherapy.xlsx
    # Nome do arquivo descompactado e utilizado como base de dados: Immunotherapy.xlsx ou direto da internet   
#==============================================================================================================================================================

todos_dados = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00428/Immunotherapy.xlsx') # Lendo o arquivo zip direto da internet

#=======

#==============================================================================================================================================================
# Separando os dados X e y
#==============================================================================================================================================================
X, y = todos_dados.drop('Result_of_Treatment', axis=1), todos_dados['Result_of_Treatment']

X = X.to_numpy() # Transformando para array
y = y.to_numpy() # Transformando para array

#==============================================================================================================================================================
# Padronizando o X para as duas formas de avaliação
#==============================================================================================================================================================
pad = StandardScaler()
X_compad = pad.fit_transform(X)
#==============================================================================================================================================================
    
for i in range(2): #Controle das duas formas de avaliação
    cont = 1 # para contagem da rodada

    rkf = RepeatedKFold(n_splits=3, n_repeats=10, random_state=100) # O n_repeats irá controlar o número de repetições

    if i == 0: #Faz a avaliação sem aninhamento
        scores_1classificador =[]
        media_scores_1classificador =[]
        scores_2classificador =[]
        media_scores_2classificador =[]
        fold = 1  

        for train_index, test_index in rkf.split(X_compad): # Retirado do site do Kfold e utilizando o X já PADRONIZADO
            print("############ "+str(fold)+ "ª FOLD SEM ANINHAMENTO ############")
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]      

            ##########################################################################
            # Validação (sem aninhamento) Padronizada utilizando LogisticRegression()
            ##########################################################################

            modelo = LogisticRegression(solver='lbfgs', max_iter=500) # Para resolver a warning que ocorria (https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter)

            modelo.fit(X_train, y_train)
            ypred = modelo.predict(X_test)
            pontuacao = metrics.accuracy_score(y_test, ypred)
            scores_1classificador.append(pontuacao) # Guarda a pontuação para tirar a média depois
            print("A pontuacao da " + str(cont) + "ª rodada na " + str(fold) + "ª fold sem aninhamento usando o LR foi de: ", pontuacao) # Imprime para dar uma olhada na pontuação da fold
            

            ##########################################################################
            # Validação (sem aninhamento) Padronizada utilizando KNeighborsClassifier()
            ##########################################################################

            modelo = KNeighborsClassifier(n_neighbors=5) # O modelo passa a ser o KNN
            modelo.fit(X_train, y_train)
            ypred = modelo.predict(X_test)
            pontuacao = metrics.accuracy_score(y_test, ypred) # Calcula a pontuação
            scores_2classificador.append(pontuacao) # Guarda a pontuação para tirar a média depois
            print("A pontuacao da " + str(cont) + "ª rodada na " + str(fold) + "ª fold sem aninhamento usando o KNN foi de: ", pontuacao) # Imprime para dar uma olhada na pontuação da fold

            fold = fold + 1 #Incrementa para contagem da fold

        media_scores_1classificador = np.mean(scores_1classificador) # Tira a média das pontuações do LR na primeira rodada
        print("--------------")
        print("Média dos Scores com o classificador LR na " + str(cont) + "ª rodada em todas as folds foi de: ", media_scores_1classificador)
        print("--------------")

        media_scores_2classificador = np.mean(scores_2classificador) # Tira a média das pontuações do KNN na primeira rodada
        print( "Média dos Scores com o classificador KNN na " + str(cont) + "ª rodada em todas as folds foi de: ", media_scores_2classificador)
        print("--------------")

    if i == 1: #Faz a avaliação com aninhamento
        
        scores_1classificador =[]
        media_scores_1classificador =[]
        scores_2classificador =[]
        media_scores_2classificador =[]
        fold = 1  

        for train_index, test_index in rkf.split(X): # Retirado do site do Kfold e utilizando o X SEM padronização padrinizado
            print("############ "+str(fold)+ "ª FOLD COM ANINHAMENTO ############")
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
        

            ##########################################################################
            # Validação (com aninhamento) padronizada utilizando LogisticRegression()
            ##########################################################################

            modelo = Pipeline([ 
            ("padronização", StandardScaler()),# Já faz a padronização dentro do pipeline
            ("LR", LogisticRegression(solver='lbfgs', max_iter=500))
            ])            
            
            modelo.fit(X_train, y_train)
            ypred = modelo.predict(X_test)
            pontuacao = metrics.accuracy_score(y_test, ypred)
            scores_1classificador.append(pontuacao) # Guarda a pontuação para tirar a média depois
            print("A pontuacao da " + str(cont) + "ª rodada na " + str(fold) + "ª fold com aninhamento usando o LR foi de: ", pontuacao) # Imprime para dar uma      olhada na pontuação da fold

            
            ###########################################################################
            # Validação (com aninhamento) padronizada utilizando KNeighborsClassifier()
            ###########################################################################        

            pipeline = Pipeline([ 
                ("padronização", StandardScaler()),# Já faz a padronização dentro do pipeline
                ("knn", KNeighborsClassifier(n_neighbors=5))
            ])

            ####Lista de pâmetros a ser utilizada pelo GridSearch e encontrar o melhor parâmetro
            parametros = {'knn__n_neighbors': [1,3,5,7,9,11,13]}

            modelo = GridSearchCV(pipeline, parametros, scoring='neg_mean_squared_error')
            modelo.fit(X_train, y_train)
            ypred = modelo.predict(X_test)
            pontuacao = metrics.accuracy_score(y_test, ypred)
            scores_2classificador.append(pontuacao) #Guarda os calores dos scores
            print("A pontuacao da " + str(cont) + "ª rodada na " + str(fold) + "ª fold com aninhamento usando o KNN foi de", pontuacao) # Imprime para dar uma bizoiada        
            
            fold = fold + 1
    
        cont = cont + 1

        media_scores_1classificador = np.mean(scores_1classificador) # Tira a média das pontuações do LR na primeira rodada
        print("--------------")
        print("Média dos Scores com o classificador LR na " + str(cont) + "ª rodada em todas as folds foi de: ", media_scores_1classificador)
        print("--------------")

        media_scores_2classificador = np.mean(scores_2classificador) # Tira a média das pontuações do KNN na primeira rodada
        print( "Média dos Scores com o classificador KNN na " + str(cont) + "ª rodada em todas as folds foi de: ", media_scores_2classificador)
        print("--------------")

############ 1ª FOLD SEM ANINHAMENTO ############
A pontuacao da 1ª rodada na 1ª fold sem aninhamento usando o LR foi de:  0.7333333333333333
A pontuacao da 1ª rodada na 1ª fold sem aninhamento usando o KNN foi de:  0.7333333333333333
############ 2ª FOLD SEM ANINHAMENTO ############
A pontuacao da 1ª rodada na 2ª fold sem aninhamento usando o LR foi de:  0.9333333333333333
A pontuacao da 1ª rodada na 2ª fold sem aninhamento usando o KNN foi de:  0.8
############ 3ª FOLD SEM ANINHAMENTO ############
A pontuacao da 1ª rodada na 3ª fold sem aninhamento usando o LR foi de:  0.7666666666666667
A pontuacao da 1ª rodada na 3ª fold sem aninhamento usando o KNN foi de:  0.7666666666666667
############ 4ª FOLD SEM ANINHAMENTO ############
A pontuacao da 1ª rodada na 4ª fold sem aninhamento usando o LR foi de:  0.7666666666666667
A pontuacao da 1ª rodada na 4ª fold sem aninhamento usando o KNN foi de:  0.7333333333333333
############ 5ª FOLD SEM ANINHAMENTO ############
A pontuacao da 1ª rodada 